<a href="https://colab.research.google.com/github/SanchitaBora/Sampling_102003177/blob/main/Assignment3(102003177).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [445]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [446]:
df=pd.read_csv("https://raw.githubusercontent.com/AnjulaMehto/Sampling_Assignment/main/Creditcard_data.csv")

In [447]:
print("Number of different classes after balancing is")
class_counts = df['Class'].value_counts()
class_counts

Number of different classes after balancing is


0    763
1      9
Name: Class, dtype: int64

**Balancing the data using oversampling of minority class**

In [448]:
minority_class = class_counts.idxmin()
majority_class = class_counts.idxmax()

minority_class_df = df[df['Class'] == minority_class]
oversampled_minority_df = minority_class_df.sample(n=class_counts[majority_class], replace=True)
balanced_df = pd.concat([df, oversampled_minority_df])

balanced_df = balanced_df.sample(frac=1).reset_index(drop=True)

In [449]:
target_var = "Class"

In [450]:
print("Number of different classes after balancing is")
class_counts = balanced_df['Class'].value_counts()
class_counts

Number of different classes after balancing is


1    772
0    763
Name: Class, dtype: int64

In [451]:
num_rows, num_cols = balanced_df.shape

print("Number of rows: ", num_rows)
print("Number of columns: ", num_cols)

Number of rows:  1535
Number of columns:  31


**Forming Samples**

In [452]:
np.random.seed(42)

In [453]:
Z=1.96
E=0.05
p=0.5

num=int((pow(Z,2)*p*(1-p))/(pow(E,2)))
print("The sample size for the samples:",num)

The sample size for the samples: 384


In [454]:
# Random Sampling
sample1=[]
sample1 = balanced_df.sample(num)

In [455]:
# Stratified Sampling
sample2=[]
strata_var = "Amount"  
strata = pd.qcut(balanced_df[strata_var], 4, labels=False)
sample2 = balanced_df.groupby(strata, group_keys=False).apply(lambda x: x.sample(min(len(x), num)))

In [456]:
# Systematic Sampling
k=10
sample3=[]
sample3 = balanced_df.iloc[0::k]

In [457]:
# Cluster Sampling
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=42).fit(balanced_df)
cluster_assignments = kmeans.labels_
selected_clusters = [0, 2, 4, 5, 8]
cluster_series = pd.Series(cluster_assignments)
sample4 = balanced_df[cluster_series.isin(selected_clusters)]

In [458]:
# Convenience Sampling
sample5 = pd.concat([balanced_df.head(num), balanced_df.tail(num)])

In [459]:
sample_names = ["Sampling1", "Sampling2", "Sampling3","Sampling4","Sampling5"]
samples = [sample1,sample2,sample3,sample4,sample5]

**Applying ML models on samples**

In [460]:
models = [DecisionTreeClassifier(), LogisticRegression(), SVC(), KNeighborsClassifier(n_neighbors=2),GaussianNB()]

In [461]:
results = {}

for model in models:
    model_name = model.__class__.__name__
    model_results = []
    for sample_name, sample in zip(sample_names, samples):
        X_train, X_test, y_train, y_test = train_test_split(sample.drop(target_var, axis=1), sample[target_var], test_size=0.3, random_state=42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        model_results.append(accuracy)
    results[model_name] = model_results

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

In [462]:
results_df = pd.DataFrame.from_dict(results, orient="index", columns=sample_names)
results_df

,Sampling1,Sampling2,Sampling3,Sampling4,Sampling5
DecisionTreeClassifier,0.982759,0.995516,1.000000,0.989865,0.982684
LogisticRegression,0.844828,0.926009,0.893617,0.956081,0.870130
SVC,0.620690,0.695067,0.723404,0.750000,0.744589
KNeighborsClassifier,0.965517,0.995516,0.978723,0.993243,1.000000
GaussianNB,0.620690,0.793722,0.723404,0.695946,0.753247


**Finding maximum value for each model**

In [463]:
max_row_values = results_df.apply(lambda x: (x.idxmax(), x.max()), axis=1)
for index, value in max_row_values.items():
    print(f"Maximum accuracy for {index} is {value[1]} which is given by: {value[0]}")

Maximum accuracy for DecisionTreeClassifier is 1.0 which is given by: Sampling3
Maximum accuracy for LogisticRegression is 0.956081081081081 which is given by: Sampling4
Maximum accuracy for SVC is 0.75 which is given by: Sampling4
Maximum accuracy for KNeighborsClassifier is 1.0 which is given by: Sampling5
Maximum accuracy for GaussianNB is 0.7937219730941704 which is given by: Sampling2
